<a href="https://colab.research.google.com/github/wuyi010/AlexNet/blob/main/%E2%80%9CAlexNet_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1、torch.nn包中主要包含了用来搭建各个层的模块  （Modules），比如全连接、二维卷积、池化等；torch.nn包中还包含了一系列有用的loss函数，这些函数也是在训练神经网络时必不可少的，比如CrossEntropyLoss、MSELoss等；另外，torch.nn.functional子包中包含了常用的激活函数，如relu、leaky_relu、prelu、sigmoid等。

2、torch.optim包则主要包含了用来更新参数的优化算法，比如SGD、AdaGrad、RMSProp、 Adam等。https://zhuanlan.zhihu.com/p/208178763

In [ ]:
import time
import torch
from torch import nn, optim
import torchvision
import sys

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)   # cuda
print("sys.executable")
sys.executable   # /usr/bin/python3
# print("sys.modules")
# sys.modules    #注意区别sys.modules和sys.builtin_module_names——前者的关键字（keys）列出的是导入的模块名，而后者则是解释器内置的模块名。
# print("sys.builtin_module_names")
# sys.builtin_module_names


cuda
sys.executable


'/usr/bin/python3'

数据下载

In [ ]:

def load_data_fashion_mnist(batch_size, resize=None, root='~/Datasets/FashionMNIST'): # root：将数据保存在本地什么位置
    if sys.platform.startswith('win'):
        num_workers = 0
    else:            # 将num_workers分配给数据加载程序的准则 https://www.cnblogs.com/hesse-summer/p/11343870.html
        num_workers = 4 
    trans = []
    if resize:  # https://zhuanlan.zhihu.com/p/476220305
        trans.append(torchvision.transforms.Resize(size=resize))# transforms.Compose([transforms.Resize(resize),transforms.ToTensor(),])
    trans.append(torchvision.transforms.ToTensor()) # 转为Tensor类型

    transform = torchvision.transforms.Compose(trans)  
    mnist_train = torchvision.datasets.FashionMNIST(root=root, train=True, download=True, transform=transform)
    mnist_test = torchvision.datasets.FashionMNIST(root=root, train=False, download=True, transform=transform)

    train_iter = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    #训练数据集的加载器，自动将数据分割成batch，顺序随机打乱
    test_iter = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=False, num_workers=num_workers)
              

    return train_iter, test_iter

batch_size = 128
train_iter, test_iter = load_data_fashion_mnist(batch_size, resize=224)

100%|██████████| 26421880/26421880 [00:01<00:00, 15928370.23it/s]


Extracting /root/Datasets/FashionMNIST/FashionMNIST/raw/train-images-idx3-ubyte.gz to /root/Datasets/FashionMNIST/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 284841.97it/s]


Extracting /root/Datasets/FashionMNIST/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /root/Datasets/FashionMNIST/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 4892157.10it/s]


Extracting /root/Datasets/FashionMNIST/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /root/Datasets/FashionMNIST/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 3287496.50it/s]


Extracting /root/Datasets/FashionMNIST/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /root/Datasets/FashionMNIST/FashionMNIST/raw



/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__() # 继承父类并使用父类初始化方法来初始化子类 
        self.conv = nn.Sequential(
            nn.Conv2d(1, 96, 11, 4), # in_channels, out_channels, kernel_size, stride, padding
            nn.ReLU(),
            nn.MaxPool2d(3, 2), # kernel_size, stride
            # 减小卷积窗口，使用填充为2来使得输入与输出的高和宽一致，且增大输出通道数
            nn.Conv2d(96, 256, 5, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(3, 2),
            # 连续3个卷积层，且使用更小的卷积窗口。除了最后的卷积层外，进一步增大了输出通道数。
            nn.Conv2d(256, 384, 3, 1, 1),
            nn.ReLU(),
            nn.Conv2d(384, 384, 3, 1, 1),
            nn.ReLU(),
            nn.Conv2d(384, 256, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(3, 2)
        )
        self.fc = nn.Sequential(
            nn.Linear(256*5*5, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 10),
        )

    def forward(self, img):
        feature = self.conv(img)
        output = self.fc(feature.view(img.shape[0], -1))
        return output

net = AlexNet()

def evaluate_accuracy(data_iter, net, device=None):
    if device is None and isinstance(net, torch.nn.Module):
        # 如果没指定device就使用net的device
        device = list(net.parameters())[0].device
    acc_sum, n = 0.0, 0
    with torch.no_grad():
        for X, y in data_iter:
            net.eval() # 评估模式, 这会关闭dropout
            acc_sum += (net(X.to(device)).argmax(dim=1) == y.to(device)).float().sum().cpu().item()
            net.train() # 改回训练模式
            n += y.shape[0]
    return acc_sum / n


def train(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs):
    net = net.to(device)
    print("training on ", device)
    loss = torch.nn.CrossEntropyLoss()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, batch_count, start = 0.0, 0.0, 0, 0, time.time()
        for X, y in train_iter:
            X = X.to(device)
            y = y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            train_l_sum += l.cpu().item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().cpu().item()
            n += y.shape[0]
            batch_count += 1
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, time %.1f sec'
              % (epoch + 1, train_l_sum / batch_count, train_acc_sum / n, test_acc, time.time() - start))



In [ ]:
lr, num_epochs = 0.001, 100
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
train(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

training on  cuda
epoch 1, loss 0.0706, train acc 0.979, test acc 0.919, time 57.4 sec
epoch 2, loss 0.0730, train acc 0.979, test acc 0.910, time 57.7 sec
epoch 3, loss 0.0685, train acc 0.979, test acc 0.916, time 57.5 sec
epoch 4, loss 0.0882, train acc 0.975, test acc 0.914, time 57.3 sec
epoch 5, loss 0.0644, train acc 0.981, test acc 0.916, time 58.4 sec
epoch 6, loss 0.0561, train acc 0.983, test acc 0.916, time 57.2 sec
epoch 7, loss 0.0654, train acc 0.980, test acc 0.913, time 58.5 sec
epoch 8, loss 0.0704, train acc 0.979, test acc 0.914, time 57.1 sec
epoch 9, loss 0.0568, train acc 0.983, test acc 0.918, time 58.3 sec
epoch 10, loss 0.0614, train acc 0.982, test acc 0.915, time 57.1 sec
epoch 11, loss 0.0657, train acc 0.981, test acc 0.915, time 58.5 sec
epoch 12, loss 0.0625, train acc 0.981, test acc 0.916, time 57.0 sec
epoch 13, loss 0.0888, train acc 0.976, test acc 0.913, time 57.9 sec
epoch 14, loss 0.0604, train acc 0.981, test acc 0.917, time 57.4 sec
epoch 15, l